In [1]:
import torch
print(torch.cuda.is_available())
print("anc")

True
anc


In [2]:
from datasets import load_dataset


ds = load_dataset("uclanlp/TemMed-Bench", "Image Pair Selection")
# print(ds)



ds_Knowledge_corpus = load_dataset("uclanlp/TemMed-Bench", "TrainSet KnowledgeCorpus")
# print(ds_Knowledge_corpus)
#vqa and report generation


# Login using e.g. `huggingface-cli login` to access this dataset
ds_vqa = load_dataset("uclanlp/TemMed-Bench", "VQA & Report Generation")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

TestSet_ImagePairSelection.json: 0.00B [00:00, ?B/s]

Generating test split:   0%|          | 0/862 [00:00<?, ? examples/s]

TrainSet_KnowledgeCorpus.json:   0%|          | 0.00/4.48M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17144 [00:00<?, ? examples/s]

TestSet_VQA_ReportGeneration.json: 0.00B [00:00, ?B/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [3]:
%pip install -q transformers accelerate bitsandbytes datasets scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.5 MB/s eta 0:00:00:00:0100:01


In [8]:
import torch
from transformers import (
    AutoProcessor, 
    LlavaNextForConditionalGeneration, 
    BitsAndBytesConfig, 
    AutoTokenizer, 
    AutoModelForCausalLM
)
from datasets import load_dataset
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from PIL import Image
import os


DEMO_MODE = True  


BASE_IMAGE_DIR = "./" 


print("models loading")
quant_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_compute_dtype=torch.float16
)

# Stage 1 for the   Eyes (LLaVA-v1.6)
print("Loading Vision Model...")
vision_processor = AutoProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")
vision_model = LlavaNextForConditionalGeneration.from_pretrained(
    "llava-hf/llava-v1.6-mistral-7b-hf", 
    quantization_config=quant_config, 
    device_map="auto"
)
# Load model directly

# Stage 2: The Brain (Phi-3-Mini)
print("Loading Logic Model...")
text_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
text_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct", 
    quantization_config=quant_config, 
    device_map="auto"
)
if text_tokenizer.pad_token is None: 
    text_tokenizer.pad_token = text_tokenizer.eos_token


def generate_medical_caption(image, model, processor):
    
    prompt = "[INST] <image>\nAnalyze this chest X-ray for: Atelectasis, Cardiomegaly, Consolidation, Edema, Pleural Effusion. Describe findings concisely. [/INST]"
    inputs = processor(text=prompt, images=image, return_tensors="pt").to("cuda")
    out = model.generate(**inputs, max_new_tokens=100)
    return processor.decode(out[0], skip_special_tokens=True).split("[/INST]")[-1].strip()

def reason_about_change(old_cap, new_cap, question, model, tokenizer):
    """Stage 2: Logic ."""
    msg = [{"role": "user", "content": f"Compare these reports:\nOLD: {old_cap}\nNEW: {new_cap}\nQUESTION: {question}\nAnswer 'Yes' or 'No'."}]
    inp = tokenizer.apply_chat_template(msg, return_tensors="pt", add_generation_prompt=True, return_dict=True).to("cuda")
    out = model.generate(input_ids=inp['input_ids'], attention_mask=inp['attention_mask'], max_new_tokens=10, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(out[0][inp['input_ids'].shape[-1]:], skip_special_tokens=True)

def load_local_image(path_str):
    try:
        return Image.open(os.path.join(BASE_IMAGE_DIR, path_str)).convert("RGB")
    except:
        return None


print("Loading VQA Dataset...")
ds = load_dataset("uclanlp/TemMed-Bench", "VQA & Report Generation", split="test")

preds, truths = [], []

print("Starting Evaluation Pipeline...")

for i, item in enumerate(tqdm(ds.select(range(100)))):
    try:

        vqa_raw = item['VQA_Final']

        vqa_list = vqa_raw if isinstance(vqa_raw, list) else [vqa_raw]


        if DEMO_MODE:
            
            cap_hist = "The lungs show mild interstitial edema and cardiomegaly."
            cap_curr = "The edema has resolved. Heart size is stable."
        else:
            
            img_h = load_local_image(item['historical_image_path'])
            img_c = load_local_image(item['image_path'])
            
            if img_h is None or img_c is None:
                # print(f"Skipping {i}: Images not found.")
                continue
                
            cap_hist = generate_medical_caption(img_h, vision_model, vision_processor)
            cap_curr = generate_medical_caption(img_c, vision_model, vision_processor)

        
        for qa in vqa_list:
            # CORRECT KEYS APPLIED HERE
            q_text = qa['VQA_question']
            a_text = qa['VQA_answer']
            
            
            pred = reason_about_change(cap_hist, cap_curr, q_text, text_model, text_tokenizer)
            
            
            p_clean = "yes" if "yes" in pred.lower() else "no"
            t_clean = "yes" if "yes" in a_text.lower() else "no"
            
            preds.append(p_clean)
            truths.append(t_clean)
            
            # Print sample output to confirm it works
            if len(preds) == 1:
                print(f"\n[SAMPLE SUCCESS]")
                print(f"Question: {q_text}")
                print(f"Prediction: {p_clean} | Truth: {t_clean}")

    except Exception as e:
        print(f"Error on sample {i}: {e}")


if len(preds) > 0:
    print(f"\nResults on {len(preds)} questions:")
    print(f"Accuracy: {accuracy_score(truths, preds)*100:.2f}%")
    print(f"F1 Score: {f1_score(truths, preds, pos_label='yes')*100:.2f}%")
else:
    print("No predictions made. Check DEMO_MODE or Image Paths.")

Loading Models...
Loading Vision Model...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading Logic Model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading VQA Dataset...
Starting Evaluation Pipeline...


  0%|          | 0/100 [00:00<?, ?it/s]


[SAMPLE SUCCESS]
Question: Is there any change noted in the position of the right internal jugular central venous catheter?
Prediction: no | Truth: no


100%|██████████| 100/100 [01:09<00:00,  1.44it/s]


Results on 264 questions:
Accuracy: 50.76%
F1 Score: 27.78%


In [17]:
# For CUDA 11.8+
!pip install bitsandbytes

# Or for CUDA 12.0+
!pip install bitsandbytes>=0.41.0

In [4]:
import torch
from transformers import (
    AutoProcessor,
    LlavaForConditionalGeneration,
    BitsAndBytesConfig,
    AutoTokenizer,
    AutoModelForCausalLM
)
from datasets import load_dataset
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from PIL import Image
import os

# ==========================================
# 1. CONFIGURATION
# ==========================================
# Set DEMO_MODE = True if you don't have the 500GB CheXpert images downloaded.
# It will simulate the vision part so you can test the Logic Model immediately.
DEMO_MODE = True

# If you HAVE the images, set this to your folder path
BASE_IMAGE_DIR = "./"

# ==========================================
# 2. LOAD MODELS
# ==========================================
print("Loading Models...")
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Stage 1: The Eyes (LLaVA-Med)
print("Loading Vision Model (LLaVA-Med)...")
vision_processor = AutoProcessor.from_pretrained("chaoyinshe/llava-med-v1.5-mistral-7b-hf")
vision_model = LlavaForConditionalGeneration.from_pretrained(
    "chaoyinshe/llava-med-v1.5-mistral-7b-hf",
    quantization_config=quant_config,
    device_map="auto"
)
print("✓ Vision Model loaded")

# Stage 2: The Brain (Phi-3-Mini)
print("Loading Logic Model (Phi-3)...")
text_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
text_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    quantization_config=quant_config,
    device_map="auto"
)
if text_tokenizer.pad_token is None:
    text_tokenizer.pad_token = text_tokenizer.eos_token
print("✓ Logic Model loaded")

# ==========================================
# 3. HELPER FUNCTIONS
# ==========================================
def generate_medical_caption(image, model, processor):
    """Stage 1: Generate medical description using LLaVA-Med."""
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": "Analyze this chest X-ray for: Atelectasis, Cardiomegaly, Consolidation, Edema, Pleural Effusion. Describe findings concisely."}
            ]
        }
    ]
    
    prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(images=image, text=prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.inference_mode():
        out = model.generate(**inputs, max_new_tokens=100)
    
    response = processor.decode(out[0], skip_special_tokens=True)
    
    # Extract assistant's response
    if "[/INST]" in response:
        return response.split("[/INST]")[-1].strip()
    elif "ASSISTANT:" in response:
        return response.split("ASSISTANT:")[-1].strip()
    return response

def reason_about_change(old_cap, new_cap, question, model, tokenizer):
    """Stage 2: Logic comparison using Phi-3."""
    msg = [
        {
            "role": "user",
            "content": f"Compare these reports:\nOLD: {old_cap}\nNEW: {new_cap}\nQUESTION: {question}\nAnswer 'Yes' or 'No'."
        }
    ]
    
    inp = tokenizer.apply_chat_template(
        msg,
        return_tensors="pt",
        add_generation_prompt=True,
        return_dict=True
    ).to(model.device)
    
    with torch.inference_mode():
        out = model.generate(
            input_ids=inp['input_ids'],
            attention_mask=inp['attention_mask'],
            max_new_tokens=10,
            pad_token_id=tokenizer.eos_token_id
        )
    
    return tokenizer.decode(out[0][inp['input_ids'].shape[-1]:], skip_special_tokens=True)

def load_local_image(path_str):
    """Load image from disk."""
    try:
        return Image.open(os.path.join(BASE_IMAGE_DIR, path_str)).convert("RGB")
    except:
        return None

# ==========================================
# 4. EVALUATION LOOP
# ==========================================
print("\nLoading VQA Dataset...")
ds = load_dataset("uclanlp/TemMed-Bench", "VQA & Report Generation", split="test")

preds, truths = [], []

print("Starting Two-Stage Evaluation Pipeline...")
print("Stage 1: Vision Model generates captions")
print("Stage 2: Logic Model answers questions\n")

# Testing on first 100 samples
for i, item in enumerate(tqdm(ds.select(range(100)), desc="Processing")):
    try:
        # Get Data
        vqa_raw = item['VQA_Final']
        # Handle if it's a list or single dict
        vqa_list = vqa_raw if isinstance(vqa_raw, list) else [vqa_raw]

        # --- STAGE 1: VISION ---
        if DEMO_MODE:
            # Simulation for testing Logic
            cap_hist = "The lungs show mild interstitial edema and cardiomegaly."
            cap_curr = "The edema has resolved. Heart size is stable."
        else:
            # Real Image Loading
            img_h = load_local_image(item['historical_image_path'])
            img_c = load_local_image(item['image_path'])
            
            if img_h is None or img_c is None:
                continue
                
            cap_hist = generate_medical_caption(img_h, vision_model, vision_processor)
            cap_curr = generate_medical_caption(img_c, vision_model, vision_processor)

        # --- STAGE 2: LOGIC ---
        for qa in vqa_list:
            q_text = qa['VQA_question']
            a_text = qa['VQA_answer']
            
            # Run Logic Model
            pred = reason_about_change(cap_hist, cap_curr, q_text, text_model, text_tokenizer)
            
            # Normalize Answers
            p_clean = "yes" if "yes" in pred.lower() else "no"
            t_clean = "yes" if "yes" in a_text.lower() else "no"
            
            preds.append(p_clean)
            truths.append(t_clean)
            
            # Print first sample to confirm it works
            if len(preds) == 1:
                print(f"\n{'='*60}")
                print(f"[SAMPLE SUCCESS - TWO-STAGE PIPELINE]")
                print(f"{'='*60}")
                print(f"Historical Caption: {cap_hist}")
                print(f"Current Caption: {cap_curr}")
                print(f"Question: {q_text}")
                print(f"Prediction: {p_clean} | Ground Truth: {t_clean}")
                print(f"{'='*60}\n")

    except Exception as e:
        print(f"Error on sample {i}: {e}")

print(f"\n{'='*60}")
print("EVALUATION RESULTS")
print(f"{'='*60}")

if len(preds) > 0:
    print(f"Total Questions Answered: {len(preds)}")
    print(f"Accuracy: {accuracy_score(truths, preds)*100:.2f}%")
    print(f"F1 Score: {f1_score(truths, preds, pos_label='yes')*100:.2f}%")
    print(f"{'='*60}")
else:
    print("No predictions made. Check DEMO_MODE or Image Paths.")
    print(f"{'='*60}")

print("\nPipeline Architecture:")
print("  Stage 1 (Vision): LLaVA-Med → Medical Captions")
print("  Stage 2 (Logic): Phi-3 → Temporal Reasoning")
print("\nTo run with real images:")
print("  1. Set DEMO_MODE = False")
print("  2. Set BASE_IMAGE_DIR to your image folder")
print("  3. Ensure image paths in dataset match your structure")

Loading Models...
Loading Vision Model (LLaVA-Med)...


processor_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

✓ Logic Model loaded

Loading VQA Dataset...
Starting Two-Stage Evaluation Pipeline...
Stage 1: Vision Model generates captions
Stage 2: Logic Model answers questions



Processing:   0%|          | 0/100 [00:00<?, ?it/s]


[SAMPLE SUCCESS - TWO-STAGE PIPELINE]
Historical Caption: The lungs show mild interstitial edema and cardiomegaly.
Current Caption: The edema has resolved. Heart size is stable.
Question: Is there any change noted in the position of the right internal jugular central venous catheter?
Prediction: no | Ground Truth: no



Processing: 100%|██████████| 100/100 [01:07<00:00,  1.47it/s]


EVALUATION RESULTS
Total Questions Answered: 264
Accuracy: 50.76%
F1 Score: 27.78%

Pipeline Architecture:
  Stage 1 (Vision): LLaVA-Med → Medical Captions
  Stage 2 (Logic): Phi-3 → Temporal Reasoning

To run with real images:
  1. Set DEMO_MODE = False
  2. Set BASE_IMAGE_DIR to your image folder
  3. Ensure image paths in dataset match your structure


Loading Models...


/usr/local/lib/python3.12/dist-packages/accelerate/utils/modeling.py:1566: UserWarning: Current model requires 4648.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

: 

: 

: 

Keys found: dict_keys(['VQA_answer', 'VQA_question'])


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.8 MB/s eta 0:00:00:00:0100:01


Loading Models...
Loading LLaVA-OneVision (The Eyes)...


ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

Loading Models...
Loading Vision Model...


ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`